In [38]:
import pandas as pd

df = pd.read_csv('results.csv', index_col=0)
df.head()

,dataset,prompt,total score,accuracy,completeness,fluency,conciseness,n_few_shot,n_bootstrapped_few_shot
0,housing_1.json,You are helping users understand an ML model's...,15.8,4.0,4.0,3.8,4.0,NaN,NaN
1,housing_1.json,You are helping users who do not have experien...,15.0,4.0,3.6,3.4,4.0,NaN,NaN
2,housing_1.json,You are helping users understand an ML model's...,15.8,4.0,4.0,3.8,4.0,NaN,NaN
3,housing_2.json,You are helping users understand an ML model's...,15.0,4.0,4.0,3.0,4.0,NaN,NaN
4,housing_2.json,You are helping users who do not have experien...,15.0,4.0,4.0,3.0,4.0,NaN,NaN


In [39]:
dataset_names = {"housing_1.json": "Housing 1",
                 "housing_2.json": "Housing 2",
                 "housing_3.json": "Housing 3",
                 "mushroom_1.json": "Mushroom",
                 "student_1.json": "Student Performance",
                 "pdf_1.json": "PDF Malware"}

df['dataset'] = df['dataset'].map(dataset_names)

In [40]:
prompts_to_ids = {df['prompt'].unique()[0]: "Prompt 1", 
                  df['prompt'].unique()[1]: "Prompt 2",
                  df['prompt'].unique()[2]: "Prompt 3"}

df['prompt'] = df['prompt'].map(prompts_to_ids)

In [41]:
df["n_few_shot"].fillna(0, inplace=True)
df["n_bootstrapped_few_shot"].fillna(0, inplace=True)

df["n_few_shot"] = df["n_few_shot"].astype(int)
df["n_bootstrapped_few_shot"] = df["n_bootstrapped_few_shot"].astype(int)

df["n_few_shot"].fillna(0, inplace=True)
df["n_bootstrapped_few_shot"].fillna(0, inplace=True)
df.head()

,dataset,prompt,total score,accuracy,completeness,fluency,conciseness,n_few_shot,n_bootstrapped_few_shot
0,Housing 1,Prompt 1,15.8,4.0,4.0,3.8,4.0,0,0
1,Housing 1,Prompt 2,15.0,4.0,3.6,3.4,4.0,0,0
2,Housing 1,Prompt 3,15.8,4.0,4.0,3.8,4.0,0,0
3,Housing 2,Prompt 1,15.0,4.0,4.0,3.0,4.0,0,0
4,Housing 2,Prompt 2,15.0,4.0,4.0,3.0,4.0,0,0


In [42]:
column_names = {"dataset": "Dataset",
                "prompt": "Prompt",
                "n_few_shot": "$L$",
                "n_bootstrapped_few_shot": "$B$",
                "accuracy": "Accuracy",
                "completeness": "Completeness",
                "fluency": "Fluency",
                "conciseness": "Conciseness",
                "total score": "Total score"}

df = df.rename(columns=column_names)
df = df[["Dataset", "Prompt", "$L$", "$B$", "Accuracy", "Completeness", "Fluency", "Conciseness", "Total score"]]

# round all scores to 3 decimal places
df = df.round(3)

df.to_csv("cleaned_results.csv")

df.head()

,Dataset,Prompt,$L$,$B$,Accuracy,Completeness,Fluency,Conciseness,Total score
0,Housing 1,Prompt 1,0,0,4.0,4.0,3.8,4.0,15.8
1,Housing 1,Prompt 2,0,0,4.0,3.6,3.4,4.0,15.0
2,Housing 1,Prompt 3,0,0,4.0,4.0,3.8,4.0,15.8
3,Housing 2,Prompt 1,0,0,4.0,4.0,3.0,4.0,15.0
4,Housing 2,Prompt 2,0,0,4.0,4.0,3.0,4.0,15.0


In [43]:
df_overall = df.groupby(["Prompt", "$L$", "$B$"]).agg({"Accuracy": "mean", "Completeness": "mean", "Fluency": "mean", "Conciseness": "mean", "Total score": "mean"}).reset_index()

df_overall = df_overall.round(3)

df_overall.to_csv("cleaned_results_overall.csv")
df_overall.head()

,Prompt,$L$,$B$,Accuracy,Completeness,Fluency,Conciseness,Total score
0,Prompt 1,0,0,4.000,4.000,3.033,4.0,15.033
1,Prompt 1,0,3,3.867,4.000,3.167,4.0,15.033
2,Prompt 1,0,5,3.867,4.000,3.300,4.0,15.167
3,Prompt 1,1,0,3.867,3.800,3.500,4.0,15.167
4,Prompt 1,3,0,3.867,3.733,3.433,4.0,15.033


In [45]:
df_overall.to_latex(buf="results_overall.tex", index=False, float_format="%.3f", caption="Overall results for each prompt and few-shot setting.")